# NL2SQL Agent Workflow - Usage Guide

This notebook demonstrates how to use the agent_workflow implementation for converting natural language queries to SQL.

## Features
- Simple API for querying databases
- Automatic retry and error recovery
- Optional monitoring and metrics
- Support for multiple LLM providers
- Production-ready implementation

In [1]:
import sys, os

# Add project root to path
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")

Project root: c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain


## Setup and Configuration

The workflow reads configuration from environment variables. Make sure your `.env` file has:

```bash
# LLM Configuration
LLM_PROVIDER=ollama
LLM_MODEL=llama3.1:8b
LLM_TEMPERATURE=0

# Workflow Settings
MAX_RETRIES=3
VECTOR_SEARCH_K=5

# Monitoring
ENABLE_MONITORING=true
EXPORT_METRICS=false
METRICS_FILE=nl2sql_metrics.json
```

In [2]:
# Import the main function
from src.agent_workflow import run_nl2sql_query, WorkflowConfig

# Display current configuration
print("Current Configuration:")
print(f"  LLM Provider: {WorkflowConfig.LLM_PROVIDER}")
print(f"  LLM Model: {WorkflowConfig.LLM_MODEL}")
print(f"  Max Retries: {WorkflowConfig.MAX_RETRIES}")
print(f"  Monitoring Enabled: {WorkflowConfig.ENABLE_MONITORING}")
print(f"  Vector Search K: {WorkflowConfig.VECTOR_SEARCH_K}")

c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'DatabaseUser'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\.venv\lib\site-packages\langchain_community\utilities\sql_database.py:159: SAWarning: Did not recognize type 'sysname' of column 'Event'
  self._metadata.reflect(
c:\Users\sourabh.gupta\Documents\Workspace\P

Current Configuration:
  LLM Provider: ollama
  LLM Model: llama3.1:8b
  Max Retries: 3
  Monitoring Enabled: True
  Vector Search K: 5


## Basic Usage

The simplest way to use the workflow is with the `run_nl2sql_query()` function.

In [3]:
# Simple query example
query = "How many customers do we have?"

result = run_nl2sql_query(query)

print("\n" + "=" * 80)
print("FINAL RESPONSE:")
print("=" * 80)
print(result["final_response"])

INFO: Pipeline started at 03:02:11
INFO: Executing query: How many customers do we have?
INFO: Parallel Schema Retrieval: Starting vector search & database lookup...
INFO: [Vector] Searching for relevant tables...
INFO: [Database] Fetching available tables...
INFO: Found 71 tables
INFO: [Database] Found 71 tables (schema: HumanResources)
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\src\agents\vector_retriver_tools.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  return Chroma(
INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: Parallel executi


FINAL RESPONSE:
We have **19,820** customers.


## Accessing Query Details

The result contains comprehensive information about the query execution.

In [4]:
# Access different parts of the result
print("\n" + "=" * 80)
print("QUERY DETAILS:")
print("=" * 80)

print("\nGenerated SQL Query:")
print("-" * 80)
print(result.get("generated_query", "N/A"))

print("\nRelevant Tables Used:")
print("-" * 80)
print(", ".join(result.get("relevant_tables", [])))

print("\nRetry Count:")
print("-" * 80)
print(result.get("retry_count", 0))

if result.get("execution_error"):
    print("\nExecution Error:")
    print("-" * 80)
    print(result["execution_error"])


QUERY DETAILS:

Generated SQL Query:
--------------------------------------------------------------------------------
SELECT COUNT(CustomerID) FROM Sales.Customer

Relevant Tables Used:
--------------------------------------------------------------------------------
Sales.Customer

Retry Count:
--------------------------------------------------------------------------------
0


## Example 1: Simple Aggregation Query

In [5]:
query = "What is the total number of orders placed?"

result = run_nl2sql_query(query)

print("\nUser Question:")
print(f"  {query}")

print("\nGenerated SQL:")
print(f"  {result['generated_query']}")

print("\nFinal Answer:")
print(f"  {result['final_response']}")

INFO: Pipeline started at 03:03:10
INFO: Executing query: What is the total number of orders placed?
INFO: Parallel Schema Retrieval: Starting vector search & database lookup...
INFO: [Vector] Searching for relevant tables...
INFO: [Database] Fetching available tables...
INFO: Found 71 tables
INFO: [Database] Found 71 tables (schema: HumanResources)
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: Parallel execution completed
INFO: Using LLM to filter relevant tables from 71 available
INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO: Fetching schema for 2 table(s)...
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\src\agents\tools.py:122: SAWarning: Did not recognize type 'Flag' of column 'OnlineOrderFlag'
  columns = self.inspector.get_columns(table, schema=schema_name)
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\src\agents\tools.py:122: SAWarning: Did not recognize type 'Or


User Question:
  What is the total number of orders placed?

Generated SQL:
  SELECT COUNT(DISTINCT SalesOrderID) FROM Sales.SalesOrderHeader

Final Answer:
  There are **31,465** unique orders placed.


## Example 2: Join Query with Filtering

In [ ]:
query = "List the top 5 products by sales amount"

result = run_nl2sql_query(query)

print("\nUser Question:")
print(f"  {query}")

print("\nGenerated SQL:")
print(f"  {result['generated_query']}")

print("\nFinal Answer:")
print(f"  {result['final_response']}")

## Example 3: Complex Query with Multiple Tables

In [ ]:
query = "Show me the departments with more than 10 employees"

result = run_nl2sql_query(query)

print("\nUser Question:")
print(f"  {query}")

print("\nGenerated SQL:")
print(f"  {result['generated_query']}")

print("\nFinal Answer:")
print(f"  {result['final_response']}")

## Example 4: Time-Based Query

In [ ]:
query = "What were the total sales in the last month?"

result = run_nl2sql_query(query)

print("\nUser Question:")
print(f"  {query}")

print("\nGenerated SQL:")
print(f"  {result['generated_query']}")

print("\nFinal Answer:")
print(f"  {result['final_response']}")

## Custom Retry Configuration

You can override the default max retries for specific queries.

In [ ]:
query = "Show me the top 10 customers by purchase frequency"

# Use custom retry limit
result = run_nl2sql_query(query, max_retries=5)

print("\nQuery with custom retry limit (5):")
print(f"  {query}")

print(f"\nRetries used: {result.get('retry_count', 0)}")
print(f"\nFinal Answer:\n  {result['final_response']}")

## Error Handling

The workflow handles errors gracefully and provides informative messages.

In [6]:
# Try a query that might fail
query = "Show me all the unicorns in the database"

try:
    result = run_nl2sql_query(query)
    
    if result.get("execution_error") or result.get("validation_error"):
        print("\nQuery had issues:")
        print(f"  Validation Error: {result.get('validation_error', 'None')}")
        print(f"  Execution Error: {result.get('execution_error', 'None')}")
    
    print(f"\nFinal Response:\n  {result['final_response']}")
    
except Exception as e:
    print(f"\nException occurred: {str(e)}")

INFO: Pipeline started at 03:04:15
INFO: Executing query: Show me all the unicorns in the database
INFO: Parallel Schema Retrieval: Starting vector search & database lookup...
INFO: [Vector] Searching for relevant tables...
INFO: [Database] Fetching available tables...
INFO: Found 71 tables
INFO: [Database] Found 71 tables (schema: HumanResources)
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: Parallel execution completed
INFO: Using LLM to filter relevant tables from 71 available
INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO: Fetching schema for 2 table(s)...
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\src\agents\tools.py:122: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  columns = self.inspector.get_columns(table, schema=schema_name)
c:\Users\sourabh.gupta\Documents\Workspace\Projects\nldbq-langchain\src\agents\tools.py:122: SAWarning: Did not recognize ty


Final Response:
  There are **no unicorns** in the database. The query returned 43 records of people with different names and characteristics, but none of them match the mythical creature known as a unicorn.


## Monitoring and Metrics

When monitoring is enabled, you can access detailed performance metrics.

In [7]:
# Check if monitoring is enabled
if WorkflowConfig.ENABLE_MONITORING:
    from src.agent_workflow.workflow import monitor
    
    print("Monitoring is enabled!")
    print("\nTo see detailed metrics, check the logs above.")
    print("The workflow automatically prints a summary after each query.")
    
    # Get the last metrics summary
    summary = monitor.get_summary()
    
    print("\n" + "=" * 80)
    print("QUICK METRICS SUMMARY:")
    print("=" * 80)
    
    print(f"\nPipeline Duration: {summary['pipeline']['total_duration']:.2f}s")
    print(f"Success: {summary['pipeline']['success']}")
    print(f"Total Steps: {summary['pipeline']['total_steps']}")
    print(f"Retries: {summary['pipeline']['retries']}")
    
    print("\nNode Execution Times:")
    for node, metrics in summary['nodes'].items():
        print(f"  {node}: {metrics['avg_time']:.3f}s avg ({metrics['executions']} exec)")
    
    print(f"\nLLM Calls: {summary['llm']['total_calls']}")
    if summary['llm']['total_tokens'] > 0:
        print(f"Total Tokens: {summary['llm']['total_tokens']:,}")
        print(f"Estimated Cost: ${summary['llm']['estimated_cost']:.4f}")
    
else:
    print("Monitoring is disabled.")
    print("To enable, set ENABLE_MONITORING=true in your .env file")

Monitoring is enabled!

To see detailed metrics, check the logs above.
The workflow automatically prints a summary after each query.

QUICK METRICS SUMMARY:

Pipeline Duration: 34.74s
Success: True
Total Steps: 5
Retries: 0

Node Execution Times:
  parallel_schema_retrieval: 2.715s avg (1 exec)
  query_generation: 3.368s avg (1 exec)
  query_validation: 22.149s avg (1 exec)
  query_execution: 0.061s avg (1 exec)
  result_formatting: 6.442s avg (1 exec)

LLM Calls: 0


## Export Metrics to JSON

You can export metrics for analysis.

In [ ]:
if WorkflowConfig.ENABLE_MONITORING:
    from src.agent_workflow.workflow import monitor
    
    # Export metrics to JSON file
    metrics_file = "query_metrics.json"
    monitor.export_metrics(metrics_file)
    
    print(f"Metrics exported to: {metrics_file}")
    
    # Read and display
    import json
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    print("\nExported Metrics Structure:")
    print(f"  - Pipeline metrics: {len(metrics['pipeline'])} fields")
    print(f"  - Node metrics: {len(metrics['nodes'])} nodes")
    print(f"  - LLM metrics: {len(metrics['llm'])} fields")
    print(f"  - Execution steps: {len(metrics['steps'])} steps")

## Batch Processing Multiple Queries

You can process multiple queries in sequence.

In [ ]:
queries = [
    "How many employees work in Sales?",
    "What is the average order value?",
    "List all departments"
]

results = []

print("Processing batch queries...")
print("=" * 80)

for i, query in enumerate(queries, 1):
    print(f"\nQuery {i}/{len(queries)}: {query}")
    
    try:
        result = run_nl2sql_query(query)
        results.append({
            "query": query,
            "answer": result["final_response"],
            "sql": result["generated_query"],
            "success": True
        })
        print(f"  ✓ Success")
        
    except Exception as e:
        results.append({
            "query": query,
            "error": str(e),
            "success": False
        })
        print(f"  ✗ Failed: {str(e)}")

print("\n" + "=" * 80)
print("BATCH RESULTS SUMMARY:")
print("=" * 80)

successful = sum(1 for r in results if r["success"])
print(f"\nSuccessful: {successful}/{len(queries)}")

for i, result in enumerate(results, 1):
    print(f"\n{i}. {result['query']}")
    if result["success"]:
        print(f"   Answer: {result['answer'][:100]}...")
    else:
        print(f"   Error: {result['error']}")

## Understanding the Workflow

The workflow consists of these steps:

1. **Parallel Schema Retrieval**: Simultaneously searches vector store and database for relevant tables
2. **Query Generation**: Converts natural language to SQL using LLM
3. **Query Validation**: Multi-layer validation (safety, syntax, semantic)
4. **Query Execution**: Executes the validated SQL query
5. **Error Recovery**: If errors occur, automatically attempts to fix the query
6. **Result Formatting**: Converts SQL results to natural language response

Each step is logged and monitored (if enabled).

## Advanced: Accessing State Information

In [ ]:
query = "Show me product categories"

result = run_nl2sql_query(query)

print("Complete State Information:")
print("=" * 80)

print("\nVector Search Results:")
vector_tables = result.get("vector_retrieved_tables", [])
print(f"  Tables from vector search: {', '.join(vector_tables) if vector_tables else 'None'}")

print("\nSchema Information:")
schema_name = result.get("schema_name")
print(f"  Schema name: {schema_name or 'N/A'}")
print(f"  Relevant tables: {', '.join(result.get('relevant_tables', []))}")

print("\nValidation:")
print(f"  Validation result: {result.get('validation_result', 'N/A')}")
print(f"  Validation error: {result.get('validation_error', 'None')}")

print("\nExecution:")
print(f"  Execution error: {result.get('execution_error', 'None')}")
print(f"  Retry count: {result.get('retry_count', 0)}")

print("\nFinal Response:")
print(f"  {result.get('final_response', 'N/A')[:200]}...")

## Tips and Best Practices

### 1. Query Formulation
- Be specific and clear in your questions
- Include relevant context (time periods, categories, etc.)
- Use standard database terminology

### 2. Monitoring
- Enable monitoring in development to understand performance
- Disable in production if not needed to reduce overhead
- Export metrics for analysis and optimization

### 3. Error Handling
- Always check for `execution_error` and `validation_error` in results
- Use try-except blocks for production code
- Set appropriate `max_retries` based on query complexity

### 4. Configuration
- Adjust `MAX_RETRIES` based on your needs
- Set `VECTOR_SEARCH_K` based on database size
- Choose appropriate LLM provider and model

### 5. Performance
- Vector search provides faster table identification
- Parallel retrieval improves overall performance
- LLM choice affects speed and accuracy

## Integration Examples

### Using in a Streamlit App

```python
import streamlit as st
from src.agent_workflow import run_nl2sql_query

st.title("Database Query Assistant")

user_query = st.text_input("Ask a question about your database:")

if user_query:
    with st.spinner("Processing..."):
        result = run_nl2sql_query(user_query)
    
    st.success("Query completed!")
    st.write(result["final_response"])
    
    with st.expander("View SQL Query"):
        st.code(result["generated_query"], language="sql")
```

### Using in an API

```python
from fastapi import FastAPI, HTTPException
from src.agent_workflow import run_nl2sql_query

app = FastAPI()

@app.post("/query")
async def query_database(query: str):
    try:
        result = run_nl2sql_query(query)
        return {
            "response": result["final_response"],
            "sql": result["generated_query"],
            "success": True
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
```

## Summary

The agent_workflow provides a production-ready, configurable solution for NL2SQL queries:

✅ **Simple API**: Single function call  
✅ **Configurable**: All settings via environment variables  
✅ **Robust**: Multi-layer validation and error recovery  
✅ **Observable**: Optional monitoring and metrics  
✅ **Fast**: Parallel schema retrieval  
✅ **Safe**: Multiple safety checks before execution  

For more details, see: `src/agent_workflow/README.md`

## Next Steps

1. **Customize Configuration**: Adjust settings in `.env` for your needs
2. **Try Different Queries**: Test with your specific database
3. **Integrate**: Use in your applications (Streamlit, FastAPI, etc.)
4. **Monitor**: Enable monitoring to optimize performance
5. **Extend**: Add custom nodes or modify existing ones

Happy querying! 🚀